In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [78]:
# Import olist data
from olist.load_data import load_all_data
data = load_all_data("data/")

In [79]:
data.keys()

dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

In [80]:
orders = data['orders'].copy()

In [81]:
from olist.preprocess import transformar_columnas_datetime
orders = transformar_columnas_datetime(orders)

In [82]:
#pasarlo a modulo
from olist.preprocess import tiempo_de_espera
orders = tiempo_de_espera(orders, is_delivered = True)
orders.head()

/Users/usuario/PycharmProjects/pythonProject5/UCEMA/07-entrenamiento/olist/preprocess.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_df['tiempo_de_espera'] = (orders_df['order_delivered_customer_date'] - orders_df[


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_de_espera
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8.436574
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13.782037
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9.394213
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13.208750
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2.873877


In [83]:
# crear real vs esperado
# ver que columnas hay
# igual que en NPS si algun numero en real_vs_esperado es negativo reemplazarlo por 0
from olist.preprocess import real_vs_esperado
orders = real_vs_esperado(orders, is_delivered = True)
orders['real_vs_esperado']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
99436    0.0
99437    0.0
99438    0.0
99439    0.0
99440    0.0
Name: real_vs_esperado, Length: 96478, dtype: float64

In [84]:
orders['real_vs_esperado'].nunique()

7755

In [85]:
orders = orders.rename(
    columns={"order_estimated_delivery_date": 'tiempo_de_espera_previsto', 'order_status': "status_de_la_orden"})
orders_2 = orders[
    ['order_id', 'tiempo_de_espera', 'tiempo_de_espera_previsto', 'real_vs_esperado', 'status_de_la_orden']]
orders_2

,order_id,tiempo_de_espera,tiempo_de_espera_previsto,real_vs_esperado,status_de_la_orden
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,2017-10-18,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,2018-08-13,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,2018-09-04,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,2017-12-15,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,2018-02-26,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,2017-03-28,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,2018-03-02,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.859421,2017-09-27,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.086424,2018-02-15,0.0,delivered


### b) `puntaje_de_compra`
     retorna un dataframe con
        order_id, es_cinco_estrellas, es_una_estrella, review_score

recordar del examen `DataFrame.apply()` o tmb se puede usar`Series.map()`

In [86]:
reviews = data['order_reviews'].copy()
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [87]:
from olist.preprocess import puntaje_de_compra
puntaje_de_compra(reviews)

,order_id,es_cinco_estrellas,es_una_estrella,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


c) numero_de_productos:
 retorna un dataframe con
    order_id, numero_de_productos (numero_de_productos_por_orden)

In [88]:
items = data['order_items'].copy()
items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [89]:
items.groupby('order_id').product_id.count().reset_index()

,order_id,product_id
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [90]:
from olist.preprocess import calcular_numero_productos
#fijarse que no uso solo un dataframe
productos = calcular_numero_productos(items)
productos

,order_id,number_of_products
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


### d) `numero_de_vendedores`:
     Retorna un dataframe con
        order_id, numero_de_vendedores (total de vendedores UNICOS !!! )

HINT!:
`pd.Series.nunique()`

In [91]:
from olist.preprocess import vendedores_unicos
vendedores = vendedores_unicos(items)
vendedores

,order_id,vendedores_unicos
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


### e) `calcular_precio_y_transporte`
     Returns a DataFrame with:
        order_id, precio, valor_transporte

recordar el agg en el groupby

| `precio` 	                    | float 	| total del precio pagado por el cliente                                                                                                                |
| `valor_transporte` 	          | float 	| el valor del freight pagado por el cliente 	                                                                                                          |

In [99]:
from olist.preprocess import calcular_precio_y_transporte
precio_y_transporte = calcular_precio_y_transporte(items)
precio_y_transporte

,order_id,precio,transporte
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


### e) `distancia_al_cliente`

    Retorna un dataframe con
        order_id, distancia_al_cliente (distancia en kilometros)

usar`haversine_distance`

In [93]:
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Computa distancia entre dos pares (lat, lng)
    Ver - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [94]:
sellers = data['sellers'].copy()
sellers = sellers[['seller_id', 'seller_zip_code_prefix']]
sellers

,seller_id,seller_zip_code_prefix
0,3442f8959a84dea7ee197c632cb2df15,13023
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195
4,51a04a8a6bdcb23deccc82b0b80742cf,12914
...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111
3091,f8201cab383e484733266d1906e2fdfa,88137
3092,74871d19219c7d518d0090283e03c137,4650
3093,e603cf3fec55f8697c9059638d6c8eb5,96080


In [95]:
customers = data['customers'].copy()
customers = customers[['customer_id', 'customer_zip_code_prefix']]
customers

,customer_id,customer_zip_code_prefix
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409
1,18955e83d337fd6b2def6b18a428ac77,9790
2,4e7b3e00288586ebd08712fdd0374a03,1151
3,b2b6027bc5c5109e529d4dc6358b12c3,8775
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056
...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,3937
99437,e7b71a9017aa05c9a7fd292d714858e8,6764
99438,5e28dfe12db7fb50a4b2f691faecea5e,60115
99439,56b18e2166679b8a959d72dd06da27f9,92120


In [96]:
geo = data['geolocation'].copy()
geo = geo.groupby("geolocation_zip_code_prefix").first()
geo

,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
geolocation_zip_code_prefix,,,,
1001,-23.549292,-46.633559,sao paulo,SP
1002,-23.548318,-46.635421,sao paulo,SP
1003,-23.549032,-46.635313,sao paulo,SP
1004,-23.550116,-46.635122,sao paulo,SP
1005,-23.549819,-46.635606,sao paulo,SP
...,...,...,...,...
99960,-27.953797,-52.029641,charrua,RS
99965,-28.173892,-52.038447,agua santa,RS
99970,-28.345143,-51.876926,ciriaco,RS


In [97]:
A = pd.merge(customers, geo, how="left", left_on = "customer_zip_code_prefix", right_on= "geolocation_zip_code_prefix")

In [98]:
B = pd.merge(customers, sellers, how="left", left_on = "customer_zip_code_prefix", right_on= "seller_zip_code_prefix")

In [59]:
pd.merge(A,B, how = "left").dropna()

,customer_id,customer_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,seller_id,seller_zip_code_prefix
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,-22.987222,-47.151073,campinas,SP,8d46553a36e68f95350a200c12f8f2e2,13056.0
5,879864dab9bc3047522c92c82e1212b8,89254,-26.468503,-49.109883,jaragua do sul,SC,8cbac7e12637ed9cffa18c7875207478,89254.0
6,fd826e7cf63160e536e0908c76c3f441,4534,-23.584547,-46.674561,sao paulo,SP,bac44fa8e13424950488659b5f765c41,4534.0
8,5adf08e34b2e993982a47070956c5c65,81560,-25.496853,-49.209023,curitiba,PR,a3082f442524a1be452e3189e003b361,81560.0
9,5adf08e34b2e993982a47070956c5c65,81560,-25.496853,-49.209023,curitiba,PR,3f8034bfc6b8b4135534975d49f04c63,81560.0
...,...,...,...,...,...,...,...,...
116432,17ddf5dd5d51696bb3d7c6291687be6f,3937,-23.587901,-46.501830,são paulo,SP,6ee85be3693ed79a8e80718743d80655,3937.0
116433,e7b71a9017aa05c9a7fd292d714858e8,6764,-23.612294,-46.765787,taboao da serra,SP,8185694b58b2ba6de20fd41ae608bbea,6764.0
116435,56b18e2166679b8a959d72dd06da27f9,92120,-29.956391,-51.167614,canoas,RS,091065b789659bc3cd9af9cb3a6253ff,92120.0
116436,56b18e2166679b8a959d72dd06da27f9,92120,-29.956391,-51.167614,canoas,RS,353e21e8bf8de2722b4fae1636664b76,92120.0
